In [ ]:
from dataDownloader  import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_nyse import DB_NYSE
from db_stock import DB_Stock
from portfolio import Portfolio
from assetAllocation import AssetAllocation
from matplotlib.dates import relativedelta
from datetime import datetime, timedelta
from commonHelper import EDateType
from scipy.stats.mstats import winsorize

import datetime
import dataDownloader
import yfinance as yf;
import pandas as pd
import commonHelper
import numpy as np
import os
import warnings
import yfinance as yf
import mplfinance as mpf
import plotly.graph_objects as go

start_date = '2014-12-01'
end_date = '2025-08-01'



def plot_candlestick(ticker:str, start:str=None, end:str=None, period:str=None, freq:str="D"):
    """
    ticker: 티커 (예: 'SPY')
    start, end: 기간 지정 (예: start="2020-01-01", end="2023-01-01")
    period: 최근 기간 지정 (예: "1y", "6mo")  # start,end 대신 사용 가능
    freq: 'D' (일간), 'M' (월간)
    """
    # 데이터 다운로드
    df = yf.download(ticker, start=start, end=end, period=period, interval="1d")
    df.columns = df.columns.get_level_values(0)

    if df.empty:
        raise ValueError("데이터를 불러오지 못했습니다. 기간/티커를 확인하세요.")

    # 리샘플링 (월간으로)
    if freq == "M":
        df = df.resample("M").agg({
            "Open": "first",
            "High": "max",
            "Low": "min",
            "Close": "last",
            "Volume": "sum"
        })

    # 차트 출력
    mpf.plot(
        df,
        type="candle",
        style="yahoo",
        volume=True,
        title=f"{ticker} {freq} Candlestick",
        ylabel="Price",
        ylabel_lower="Volume"
    )


def plot_candlestick_interactive(ticker, start=None, end=None, period=None, freq="D"):
    df = yf.download(ticker, start=start, end=end, period=period, interval="1d", group_by="column")
    df.columns = df.columns.get_level_values(0)
    
    if freq == "M":
        df = df.resample("M").agg({
            "Open": "first",
            "High": "max",
            "Low": "min",
            "Close": "last",
            "Volume": "sum"
        })

    df = df.dropna()

    fig = go.Figure(data=[go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name=ticker
    )])

    fig.update_layout(
        title=f"{ticker} {freq} Candlestick",
        xaxis_rangeslider_visible=False,
        yaxis_title="Price"
    )

    fig.show()


# 2020년~2023년까지 SPY 일봉 캔들
# plot_candlestick("SPY", start="2020-01-01", end="2023-01-01", freq="D")
plot_candlestick_interactive("SPY", start="2020-01-01", end="2023-01-01", freq="D")

[*********************100%***********************]  1 of 1 completed


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed